In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
front_2Dcoord = np.load('front_image.npy', allow_pickle=True)
back_2Dcoord = np.load('back_image.npy', allow_pickle=True)
front_img = cv2.imread('front.png')

In [3]:
#visualization
print(front_2Dcoord.shape, back_2Dcoord.shape)
from copy import deepcopy
vis_img = deepcopy(front_img)
for i in range(front_2Dcoord.shape[0]):
    cv2.circle(vis_img, (int(front_2Dcoord[i,0]), int(front_2Dcoord[i,1])), 2, (255,0,0), 2)
plt.imshow(vis_img[:,:,::-1])
plt.show()

(12, 2) (12, 2)


In [4]:
# ------------TODO----------------
# Compute corresponding 3D coordinate
# ------------TODO----------------
front_3Dcoord = np.zeros((12, 3))
back_3Dcoord = np.zeros((12, 3))
convert = np.array([[0, 0], [50, 0], [50, 50], [0, 50]])
start_xy = np.array([[0, 0], [160, 240], [400, 400]])
xy_3D = (start_xy.reshape(3, 1, 2) + convert.reshape(1, 4, 2)).reshape(12, 2)

front_3Dcoord[:, :2] = xy_3D
back_3Dcoord[:, :2] = xy_3D
back_3Dcoord[:, -1] = 150

all_2Dcoord = np.concatenate((front_2Dcoord, back_2Dcoord), axis=0)
all_3Dcoord = np.concatenate((front_3Dcoord, back_3Dcoord), axis=0)
print(all_2Dcoord.shape, all_3Dcoord.shape)

(24, 2) (24, 3)


In [5]:
# ------------TODO----------------
# Construct the Equation Pm=0 and solve m
# ------------TODO----------------

P = np.zeros((48, 12))
m = np.zeros((12, 1))
for i in range(48):
    insert_part = np.append(all_3Dcoord[i//2], 1)
    if i % 2 == 0:
        P[i, :4] = insert_part
        P[i, -4:] = insert_part * all_2Dcoord[i//2, 0]
    else:
        P[i, 4:8] = insert_part
        P[i, -4:] = insert_part * all_2Dcoord[i//2, 1]
U, D, Vh_T = np.linalg.svd(P)
m = Vh_T.T[:, -1]

In [6]:
# ------------TODO----------------
# Solve K, and [R T] from m
# ------------TODO----------------
M_std = m.reshape(3, 4)
a1 = M_std[0, :3]
a2 = M_std[1, :3]
a3 = M_std[2, :3]
b = M_std[:, -1]
rho = 1 / (a3.dot(a3) ** 0.5)
cx = (rho ** 2) * np.dot(a1, a3)
cy = (rho ** 2) * np.dot(a2, a3)
a4 = np.cross(a2, a3)
a5 = np.cross(a3, a1)
a6 = np.cross(a1, a3)
cos_theta = np.dot(a4, a5) / ((a4.dot(a4) ** 0.5) * (a5.dot(a5) ** 0.5))
sin_theta = (1 - cos_theta ** 2) ** 0.5
alpha = (rho ** 2) * (a6.dot(a6) ** 0.5) * sin_theta
beta = (rho ** 2) * (a4.dot(a4) ** 0.5) * sin_theta
r1 = a4 / (a4.dot(a4) ** 0.5)
r3 = a3 / (a3.dot(a3) ** 0.5)
r2 = np.cross(r3, r1)
K = np.zeros((3, 3))
R = np.zeros((3, 3))
T = np.zeros((3, 1))

K[0, 0] = alpha
K[0, 1] = -alpha * (cos_theta / sin_theta)
K[0, 2] = cx
K[1, 1] = beta / sin_theta
K[1, 2] = cy
K[2, 2] = 1

R[0] = r1
R[1] = r2
R[2] = r3

T = rho * np.linalg.inv(K) @ b
print(T)
# 小数点两位后的差异可能是因为np.linalg.svd版本不同的差异导致的.

[383001.83767393 -67357.81803382 -67071.11348848]


In [7]:
# save results
save_dict = {'3Dcoord': all_3Dcoord, 'P':P, 'm':m, 'K':K, 'R':R, 'T':T}
np.save('../results/calibr', save_dict)

# The reference answer of T should be [383001.8386981  -67357.81794783  67071.11338303]
# The results are very strange because the data is fake.